In [6]:
import pandas as pd

# USE CASE FOR LOOP EMBEDDING QUESTIONS FOR TESTING 
intrebari_test = []
df = pd.read_csv("questions_penal_code_ro.csv")
question_and_context = df.to_json(orient="records")

for ix, row in enumerate(df.values):

    intrebari_test.append(row[0])
    
    if ix >= 10:
        break
        
print(intrebari_test)

['Cum definește Codul penal forma de vinovăție «intenția»?', 'Ce este culpa și când atrage răspunderea penală?', 'Ce înseamnă tentativa și cum se sancționează concret?', 'În ce situație desistarea voluntară te poate scăpa de pedeapsă?', 'Care e diferența, pe scurt, între autor, instigator și complice?', 'Ce este participația improprie și când se aplică?', 'Cum se definește recidiva în Codul penal?', 'Ce presupune concursul de infracțiuni și cum se contopesc pedepsele?', 'Când vorbim despre o infracțiune continuată?', 'Cum se aplică legea penală mai favorabilă (lex mitior)?', 'În ce condiții se aplică legea penală română pentru fapte comise în străinătate?']


In [ ]:
contextsFound = [] 
def get_answer_csv_questions(intrebari):
    for ix, intrebare in enumerate(intrebari): 
        embedding = get_embedding(intrebare, model="text-embedding-3-large")
        article_answers = get_articles_milvus(query_embedding, top_k=2)
        for answer in article_answers:
             contextsFound.append({indexQuestion: ix, text: answer[text]})
       
        
        
    
    

In [ ]:
import json
import os
import re
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from pymilvus import MilvusClient, connections, DataType, Collection

load_dotenv()

MILVUS_IP = os.getenv("MILVUS_IP")
#MILVUS
#note add a .env IP variable !! 
clientMilvus = MilvusClient(
    uri="http://localhost:19530"
    # uri = f"http://{MILVUS_IP}:19530"
)

#DB
conn = connections.connect(host="localhost", port=19530)
# database = db.create_database("CodPenal")

#SCHEMA
schema = MilvusClient.create_schema(
    auto_id=True,
    enable_dynamic_field=True
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True, auto_id=True)
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=65535)
schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=3072)

# clientMilvus.drop_collection(collection_name="codPenal_collection")
# clientMilvus.drop_collection(collection_name="codPenal_collectionTest")
#

##COLLECTION
index_params = {
    "index_type": "IVF_FLAT",
    "params": {"nlist": 128},
    "metric_type": "COSINE"
}

collection = Collection(name="codPenal_collection", schema=schema)
collection.create_index("embedding", index_params)
collection.load()
# executed only once
# clientMilvus.create_collection(
#     collection_name="codPenal_collection",
#     dimension=3072,
#     schema=schema,
#     metric_type="COSINE"
# )
db_name = "codPenal_collection"

# clientMilvus.create_collection(
#     collection_name="codPenal_collectionTest",
#     dimension=3072,
#     schema=schema,
#     metric_type="COSINE"
# )
# db_name_test = "codPenal_collectionTest"



def insert_data(collectionName, data):
    """
    Inserts data into the specified Milvus collection.

    Args:
        collectionName (str): The name of the Milvus collection.
        data (list): A list of data entries to insert.

    Returns:
        None
    """
    res = clientMilvus.insert(
        collection_name=collectionName,
        data=data,
    )
    print(res)

# EMBEDING / AI
# Get the key from environment
api_key = os.getenv("OPENAI_API_KEY")

# Initialize client correctly (must be keyword arg)
clientOpenAI = OpenAI(api_key=api_key)


def get_chatbot_answer(query: str, answers: list[dict]):
    """
    Generates a concise, professional answer in Romanian to a legal query using provided articles.

    Args:
        query (str): The user's legal question.
        answers (list): List of relevant articles or answers to use for context.

    Returns:
        str: The chatbot's answer in Romanian.
    """
    context = str(answers)
    #TODO: model should be on the top in a config or something
    completion = clientOpenAI.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "Eşti un sistem profesionist de avocatură care vorbeşte limba română şi este specializat în Codul Penal şi legislaţia română."
            },
            {
                "role": "user", "content": f"""
                --IDENTITY: A romanian lawyer that answers short and concise.
                --TASK: You are going to answer the USER_QUERY by using CONTEXT.
                --USER_QUERY: {query}
                --CONTEXT: {context}
                """
            }
        ]
    )

    #TODO: 0 error handling, ex : 529 open ai 
    print(completion.choices[0].message.content)
    return completion.choices[0].message.content

def get_embedding(text, model="text-embedding-3-large"):
    """
    Generates an embedding vector for the given text using OpenAI's embedding model.

    Args:
        text (str): The input text to embed.
        model (str, optional): The embedding model to use. Defaults to "text-embedding-3-large".

    Returns:
        list: The embedding vector for the input text.
    """
    return clientOpenAI.embeddings.create(input=[text], model=model).data[0].embedding

def get_articles_milvus(query_embedding, top_k=2):
    """
    Searches the Milvus collection for articles most similar to the query embedding.

    Args:
        query_embedding (list): The embedding vector of the query.
        top_k (int, optional): Number of top results to return. Defaults to 2.

    Returns:
        list: List of dictionaries containing article id, score, and text.
    """
    res = clientMilvus.search(
        collection_name=db_name,
        data=[query_embedding],
        limit=top_k,
        search_params={"metric_type": "COSINE", "params": {}},
        output_fields=["text"]
    )

    # res is usually a list of result sets (one per query vector)
    hits = res[0]

    results = []
    for h in hits:
        # Handle both Hit API shapes
        try:
            # Classic Hit object from Collection.search
            item = {
                "id": int(getattr(h, "id", None) or getattr(h, "pk", None) or -1),
                "score": float(getattr(h, "distance", None) or getattr(h, "score", None)),
                "text": h.get("text") if hasattr(h, "get") else (h["text"] if isinstance(h, dict) else None),
            }
        except Exception:
            # MilvusClient may already return dict-like rows
            item = {
                "id": int(h.get("id", -1)) if isinstance(h, dict) else -1,
                "score": float(h.get("distance", h.get("score", 0.0))) if isinstance(h, dict) else 0.0,
                "text": (h.get("text") if isinstance(h, dict) else None),
            }
        results.append(item)

    return results

# Usage
# query = "Ce înseamnă «infracțiune» în Codul penal?"
# query_embedding = get_embedding(query)
# articles = get_articles_milvus(query_embedding, top_k=2)

# print(json.dumps(articles, ensure_ascii=False, indent=2))


# USE CASE FOR LOOP EMBEDDING QUESTIONS FOR TESTING 
intrebari_test = []
df = pd.read_csv("questions_penal_code_ro.csv")
question_and_context = df.to_json(orient="records")

for ix, row in enumerate(df.values):

    intrebari_test.append(row[0])
    
    if ix > 0:
        break
        
print(intrebari_test)


contextsFound = [] 

def get_answer_question(intrebare):
    embedding = get_embedding(intrebare, model="text-embedding-3-large")
    article_answers = get_articles_milvus(embedding, top_k=2)
    answer_from_gpt = get_chatbot_answer(intrebare, article_answers)
    print(article_answers)
    print(answer_from_gpt)


for intrebare in intrebari_test:
    get_answer_question(intrebare)






['Cum definește Codul penal forma de vinovăție «intenția»?', 'Ce este culpa și când atrage răspunderea penală?', 'Ce înseamnă tentativa și cum se sancționează concret?', 'În ce situație desistarea voluntară te poate scăpa de pedeapsă?', 'Care e diferența, pe scurt, între autor, instigator și complice?', 'Ce este participația improprie și când se aplică?', 'Cum se definește recidiva în Codul penal?', 'Ce presupune concursul de infracțiuni și cum se contopesc pedepsele?', 'Când vorbim despre o infracțiune continuată?', 'Cum se aplică legea penală mai favorabilă (lex mitior)?', 'În ce condiții se aplică legea penală română pentru fapte comise în străinătate?']


NameError: name 'query_embedding' is not defined

In [15]:
test = {'abc':12}

test_json = json.dumps(test)
print(test_json)

print(str(test))

{"abc": 12}
{'abc': 12}


In [16]:
intrebari_test = []
df = pd.read_csv("questions_penal_code_ro.csv")
question_and_context = df.to_json(orient="records")

for ix, row in enumerate(df.values):

    intrebari_test.append(row[0])
    
    # if ix > 0:
    #     break
        
print(intrebari_test)


['Cum definește Codul penal forma de vinovăție «intenția»?', 'Ce este culpa și când atrage răspunderea penală?', 'Ce înseamnă tentativa și cum se sancționează concret?', 'În ce situație desistarea voluntară te poate scăpa de pedeapsă?', 'Care e diferența, pe scurt, între autor, instigator și complice?', 'Ce este participația improprie și când se aplică?', 'Cum se definește recidiva în Codul penal?', 'Ce presupune concursul de infracțiuni și cum se contopesc pedepsele?', 'Când vorbim despre o infracțiune continuată?', 'Cum se aplică legea penală mai favorabilă (lex mitior)?', 'În ce condiții se aplică legea penală română pentru fapte comise în străinătate?', 'Când se reține legitima apărare? Ce condiții trebuie îndeplinite?', 'Ce este starea de necesitate și cum funcționează în practică?', 'Poate consimțământul persoanei vătămate să înlăture caracterul penal al faptei?', 'Ce este eroarea de fapt și ce efect are asupra răspunderii penale?', 'Când este o persoană iresponsabilă din punct d

In [13]:

def get_answer_question(intrebare):
    embedding = get_embedding(intrebare, model="text-embedding-3-large")
    article_answers = get_articles_milvus(embedding, top_k=2)
    answer_from_gpt = get_chatbot_answer(intrebare, article_answers)
    


for intrebare in intrebari_test:
    get_answer_question(intrebare)


Codul Penal român definește forma de vinovăție „intenția” în articolul 16, stabilind că fapta este săvârșită cu intenție atunci când făptuitorul:

a) prevede rezultatul faptei sale, urmărind producerea lui prin săvârșirea acelei fapte;
b) prevede rezultatul faptei sale și, deși nu-l urmărește, acceptă posibilitatea producerii lui.

Astfel, intenția implică atât conștiința rezultatelor faptei, cât și o atitudine activă sau acceptantă față de acestea.
Culpa este o formă de vinovăție prevăzută în Codul Penal Român, care se referă la faptele săvârșite fără intenție directă, dar cu neatenție sau imprudență. Răspunderea penală se atrage doar pentru faptele comise cu vinovăție, în conformitate cu articolul 15 din Codul Penal, care stipulează că infracțiunea trebuie să fie săvârșită cu vinovăție și este singurul temei al răspunderii penale. 

Conform articolului 16, culpă există în două forme: când făptuitorul prevede rezultatul faptei, dar nu-l acceptă, sau când nu-l prevede, deși ar fi trebu